In [ ]:
import numpy as np, os, time
import tensorflow as tf
from tensorflow import keras
from argparse import Namespace
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tmodel
args: Namespace = tmodel.load_args()
tf.config.set_visible_devices([], 'GPU')             # disable GPU for this notebook

signal_index=args.signal
feature_type=args.feature_type

In [ ]:
data=tmodel.get_demo_data()
signals = data['signals']
T = data['times'][signal_index]
X: np.ndarray = tmodel.get_features( T, feature_type, args )
Y = signals[signal_index]

validation_split = int(0.8*X.shape[0])
Xtrain=X[:validation_split]
Xval=X[validation_split:]
Ytrain=Y[:validation_split]
Yval=Y[validation_split:]

strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")
with strategy.scope():
    model = tmodel.create_streams_model( X.shape[1], dropout_frac=args.dropout_frac, n_streams=args.nstreams )
    model.compile( optimizer=tf.keras.optimizers.Adam( learning_rate=args.learning_rate ), loss=args.loss )

In [ ]:
ckp_file = tmodel.get_ckp_file( feature_type, signal_index)
model.load_weights(ckp_file)
p0=model.predict(Xtrain,batch_size=256)
p1=model.predict(Xval,batch_size=256)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(T,Y,label='truth')
plt.plot(T[:validation_split],p0[:,0],label='train prediction')
plt.plot(T[validation_split:],p1[:,0],label='val prediction')
plt.title(f'Signal {signal_index} (ftype={feature_type}): nfeatures={args.nfeatures}')
plt.legend()
plt.tight_layout()
plt.show()